# H2 Model

##### Author: Hrönn Kjartansdóttir
##### Methanogenesis model where H2 uptake rate is assumed to be the limiting factor for the model

In [ ]:
# Import all packages needed for the model
import numpy as np
import scipy
from scipy.integrate import solve_ivp
from scipy.optimize import least_squares

import matplotlib.pyplot as plt
import pandas as pd

import random

import os

# Set the working directory
os.chdir('C:\\Users\\hronn\\Desktop\\Thesis\\Jupyter\\reactor1') # <- Change accordingly


# - Pre work on experimental data 

In [ ]:
# Import the experimental data -> Measured pressure values over time
df = pd.read_csv("reactor1_exppressure.csv", sep=";") # <- Change accordingly
df

In [ ]:
# Sort the experimental data to get arrays of:
    # df_indices = indices of experimental pressure values grouped by fills -> df_indices[0] = indices for the first fill
    # nan_indices = nan values grouped to determine where the margin between fills lies
    # df_pressure = experimental pressure values grouped by fills
    # in_pressure = starting pressure for each fill
    # end_pressure = end pressure value for each fill
    # df_time = time each pressure value was measured grouped by fills

def grouping(dataframe, time):
            # Gather and convert all non-zero values and all nan values into lists
    df = dataframe.index.get_indexer(dataframe.index[dataframe.notnull()]).tolist()
    nan = dataframe.index.get_indexer(dataframe.index[dataframe.isnull()]).tolist()
    
            # Group indeces of nan values and non-zero pressure values -> find where the shift between fills occurs
    def indices(values):
        
        prev_number = min(values) if values else None
        indices = list()

        for number in sorted(values):
            if number != prev_number+1:
                indices.append([number])
            elif len(indices[-1]) > 1:
                indices[-1][-1] = number
            else:
                indices[-1].append(number)
            prev_number = number
            
        return indices
    
    df_indices = indices(df)
    nan_indices = indices(nan)
    
            # Group all pressure values by fills
    df_pressure = []

    def values(dataframe):
        for i in df_indices:
        
            values = dataframe.iloc[i[0]:(i[1]+1)].tolist()
            df_pressure.append(values)
            
        return df_pressure
    
    df_pressure = values(dataframe) 
    
            # Group all starting and end pressure values into respective arrays
    in_pres = []
    end_pres = []
    
    for i in range(len(df_pressure)):
    
        inn = df_pressure[i][0]
        out = df_pressure[i][-1]
    
        in_pres.append(inn)
        end_pres.append(out)
    
    
            # Group time values by fills
    df_time = []
    
    for i in df_indices:
        time_values = time.iloc[i[0]:(i[1]+1)].tolist()
        df_time.append(time_values)
        

            # Variables to be returned
    
    return df_indices, nan_indices, df_pressure, in_pres, end_pres, df_time

            # Call function:

[df_indices, nan_indices, df_pressure, in_pressure, end_pressure, df_time ] = grouping(df["pressure_bar"], df["days"])

In [ ]:
# If needing to check if time data is sorted correctly -> if not go back to experimental data and look for where the error occurred

            # Time data to test:
test_list = df_time[0]
   
            # Check sorting of the list using sort()
flag = 0
test_list1 = test_list[:]
test_list1.sort()
if (test_list1 == test_list):
    flag = 1
      
            # print the results
if (flag) :
    print ("Yes, List is sorted.")
else :
    print ("No, List is not sorted.")

In [ ]:
# Set all global variables for the model

            # Molecular weight of liquids:
M_H2  = 2.016*1e-3 # mol/m3
M_CO2 = 44.009*1e-3 # mol/m3
M_CH4 = 16.043*1e-3 # mol/m3 
M_H2O = 18.015*1e-3 # mol/m3

            # Liquid density:
density_l = 992.2 # kg/m³

            # Sandrock porosity:
pore = 0.2 # %

            # Dissolution/evaporation rate constants:
K_diss_H2 = 1.05e-2
K_diss_CO2 = 1.87e-3
K_diss_CH4 = 9.32e-5

K_evap_CH4 = 1.81e-2

            # Maximum biomass growth rate:
mu_max = 0.0427 # 1/s

            # temperature:
temperature = 273+40 # Kelvin (K)

            # max uptake rates:
r_H2_max = 1 # mmol/g*h
r_CH4_min = 0 # mmol/g*h

            # Henry solubility constants:
H_H2 = 7.13e-9*1e3
H_CO2 = 2.31e-7*1e3
H_CH4 = 1.08e-8*1e3

            # Initial mole fraction for H2 and CO2 in gas phase -> 10% H2, 2.5% CO2:
x_g_H2_th = 0.10 # <- Change accordingly
x_g_CO2_th = 0.025 # <- Change accordingly

            # Pressure in the reactor at end of the evaporation phase
pressureCH4resid = 2.06e5 # Pa


In [ ]:
#Inital guess values for parameters of interest = 10**theta0[i] 
theta0 = [0.32, 4.02, -4.56, -6.06, -2.56, -8.36] # <- Change accordingly


# - RK45() function 

In [ ]:
# RK45() to track changes in parameters over time

def rk45(t, y, U_H2, K_Monod_H2, K_Monod_CO2, d, x_l_eps):
    w_g_H2, w_g_CO2, w_g_CH4, w_l_H2, w_l_CO2, w_l_CH4, m, density_g, S_liq = y
      
            # Molecular weight of gasses:
    M_g = ((w_g_H2/M_H2) + (w_g_CO2/M_CO2) + (w_g_CH4/M_CH4))**-1
    
            # Gas pressure:
    R_gas = 8.314 / M_g
    pressure_g = density_g * R_gas * temperature

            # Mole fractions of components in gas phase:
    x_g_H2 = (w_g_H2/M_H2)*M_g
    x_g_CO2 = (w_g_CO2/M_CO2)*M_g
    x_g_CH4 = (w_g_CH4/M_CH4)*M_g

            # mass fraction of liquid H2O:
    w_l_H2O = 1 - (w_l_H2 + w_l_CO2 + w_l_CH4)

            # Molecular weight of liquids:
    M_l = ((w_l_H2/M_H2) + (w_l_CO2/M_CO2) + (w_l_CH4/M_CH4) + (w_l_H2O/M_H2O))**-1

            # Mole fractions of components in liquid phase:
    x_l_H2 = (w_l_H2/M_H2)*M_l
    x_l_CO2 = (w_l_CO2/M_CO2)*M_l
    x_l_CH4 = (w_l_CH4/M_CH4)*M_l

            # Saturated mole fraction of components in liquid phase:
    x_l_sat_H2 = (M_l/density_l) * H_H2 * x_g_H2 * pressure_g
    x_l_sat_CO2 = (M_l/density_l) * H_CO2 * x_g_CO2 * pressure_g
    x_l_sat_CH4 = (M_l/density_l) * H_CH4 * x_g_CH4 * pressure_g
       
            # Source term due to dissolution/evaporation:
    R_diss_H2  = K_diss_H2*(x_l_sat_H2 - x_l_H2)
    R_diss_CO2 = K_diss_CO2*(x_l_sat_CO2 - x_l_CO2)

    if x_l_CH4 < x_l_sat_CH4:
        R_diss_CH4 = K_diss_CH4*(x_l_sat_CH4 - x_l_CH4)
    else:
        R_diss_CH4 = K_evap_CH4*(x_l_sat_CH4 - x_l_CH4)

    R_diss_tot = R_diss_H2 + R_diss_CO2 + R_diss_CH4

            # Source for saturation of the liquid phase:
    source_S_liq = (1/(pore*density_l))*R_diss_tot
       
            # Saturation of the gas phase:
    S_gas = 1 - S_liq
       
            # Uptake/production rates of H2, CO2, CH4 (r_(i)):

                #if x_l_CO2 or x_l_H2 is smaller or equal to x_l_eps => stop consuming
    if (x_l_CO2 <= x_l_eps) or (x_l_H2 <= x_l_eps):
        r_H2 = 0
        mu = 0
    else:
        r_H2  = min(U_H2* x_l_H2 , r_H2_max) # mol of hydrogen / kg of dry biomass / hour
        mu_monod = mu_max * (x_l_H2 / (K_Monod_H2 + x_l_H2)) * (x_l_CO2 / (K_Monod_CO2 + x_l_CO2))
        mu = min(mu_monod, (r_H2 * mu_max / r_H2_max))      

    r_CO2 = r_H2 / 4
    r_CH4 = r_CO2 - (5.855 * mu)

            # Biomass change over time:
    source_m = ((mu - d) * m)/3600 # divide by 3600 to get all units in si units (from hour to seconds)

            # Source term for production/uptake rates:
    R_uptake_H2 = (-r_H2*m*M_H2*S_liq*pore)/3600
    R_uptake_CO2 = (-r_CO2*m*M_CO2*S_liq*pore)/3600
    R_uptake_CH4 = (r_CH4*m*M_CH4*S_liq*pore)/3600

            # Mass fractions change over time:
    source_w_l_H2 = 1/(pore*density_l*S_liq) * (R_uptake_H2 + R_diss_H2 - (w_l_H2 * R_diss_tot))
    source_w_l_CO2 = 1/(pore*density_l*S_liq) * (R_uptake_CO2 + R_diss_CO2 - (w_l_CO2 * R_diss_tot))
    source_w_l_CH4 = 1/(pore*density_l*S_liq) * (R_uptake_CH4 + R_diss_CH4 - (w_l_CH4 * R_diss_tot))

            # Gas density change over time:
    source_density_g = -(R_diss_tot / (pore * S_gas)) * (1 - (density_g / density_l))
        
             # Mass fraction change over time:
    source_w_g_H2 = -(1/(pore*density_g*S_gas)) * (R_diss_H2 - w_g_H2 * R_diss_tot) 
    source_w_g_CO2 = -(1/(pore*density_g*S_gas)) * (R_diss_CO2 - w_g_CO2 * R_diss_tot) 
    source_w_g_CH4 = -(1/(pore*density_g*S_gas)) * (R_diss_CH4 - w_g_CH4 * R_diss_tot) 
 
            # Variables to be returned:
    return [source_w_g_H2, source_w_g_CO2, source_w_g_CH4, source_w_l_H2, source_w_l_CO2, source_w_l_CH4, source_m, source_density_g, source_S_liq]

# - Pressure simulation functions -> pressure_fill1(), pressure_otherfills(), simulated_pressure()

In [ ]:
# Pressure generation function for fill 1

def pressure_fill1(time, in_pressure, theta):
     
            # Initial mass fraction of components in liquid phase:
    w_l_H2_in  = 0
    w_l_CO2_in = 0
    w_l_CH4_in = 0
       
            # Initial Saturation of the liquid phase:
    S_liq_in = 0.35 
    
            # Initial gas pressure:
    pressure_g_in = in_pressure*1e5 #Pa
    
            # Initial mole fraction of components in gas phase:
    x_g_H2_in  = (pressure_g_in - 1e5) / pressure_g_in * x_g_H2_th
    x_g_CO2_in = (pressure_g_in - 1e5) / pressure_g_in * x_g_CO2_th 
    x_g_CH4_in = 1 - x_g_H2_in - x_g_CO2_in
    
            # Initial molecular mass of all gasses:
    M_g = (x_g_H2_in * M_H2) + (x_g_CO2_in * M_CO2) + (x_g_CH4_in * M_CH4)

            # Initial gas density:
    R_gas_in = 8.314 / M_g
    density_g_in = pressure_g_in / R_gas_in / temperature

            # Initial mass fraction of components in gas phase:
    w_g_H2_in  = (x_g_H2_in * M_H2) / M_g
    w_g_CO2_in = (x_g_CO2_in * M_CO2) / M_g
    w_g_CH4_in = (x_g_CH4_in * M_CH4) / M_g
    
            # Time array in seconds and end time:
    time_sim = np.array(time) - np.array(time[0])
    time_sim = time_sim * 24 * 60 * 60                
    time_end = time_sim[-1]
    
            # Initial values for the parameters of interest:
    BM_in = 10**theta[0]
    U_H2 = 10**theta[1]
    K_Monod_H2 = 10**theta[2]
    K_Monod_CO2 = 10**theta[3]
    d = 10**theta[4]
    x_l_eps = 10**theta[5]

            # Solve_ivp to solve for RK45():
    sol = solve_ivp(rk45, [0,time_end],y0=[w_g_H2_in, w_g_CO2_in, w_g_CH4_in, w_l_H2_in, w_l_CO2_in, w_l_CH4_in, BM_in, density_g_in, S_liq_in], 
                    args=(U_H2, K_Monod_H2, K_Monod_CO2, d, x_l_eps), t_eval=time_sim, atol=1e-8, rtol=1e-6)
    
            # Save the tracked and returned values from solve_ivp():
    w_g_H2 = sol.y[0]
    w_g_CO2 = sol.y[1]
    w_g_CH4 = sol.y[2]
    w_l_H2 = sol.y[3] 
    w_l_CO2 = sol.y[4]
    w_l_CH4 = sol.y[5]
    biomass = sol.y[6]
    density_g = sol.y[7]
    S_liq = sol.y[8]

            # Calculate the new initial saturation of liquid phase to use as input for the next fill:
    S_liq_in_new = S_liq[-1] * (1 - (1 - 0.557) * w_l_CH4[-1] - w_l_H2[-1] - w_l_CO2[-1])
            
            # Calculate the new initial molecular weight of gasses:
    M_g_new = ((w_g_H2 / M_H2) + (w_g_CO2 / M_CO2) + (w_g_CH4 / M_CH4))**-1
    
            # Calculate the pressure change over time:
    R_gas = 8.314 / M_g_new
    pressure_g_bar = density_g * R_gas * temperature * 1e-5 #to change from pascal to bar * 1e-5
    pressure_g_pascal = density_g * R_gas * temperature

            # Calculate the initial mass fraction of CH4 to input into next fill:
    w_l_CH4_in_new = w_l_CH4[-1] * 0.557 # 55.7% of the end value go to the next fill
   
            # Calculate the initial biomass to input into next fill:
    BM_in_new = biomass[-1] /(1 - (1 - 0.557) * w_l_CH4[-1] - w_l_H2[-1] - w_l_CO2[-1])

            # Variables to be returned:
    return w_l_CH4_in_new, pressure_g_pascal, S_liq_in_new, BM_in_new, biomass

In [1]:
# Pressure generation function for all fills except 1

def pressure_otherfills(time, in_pressure, theta, w_l_CH4_in_new, S_liq_in, BM_in_new):

            # Initial mass fraction of H2 and CO2 in liquid phase:
    w_l_H2_in  = 0
    w_l_CO2_in = 0
    
             # Initial gas pressure:
    pressure_g_in = in_pressure*1e5 # Pa

            # Initial mole fraction of components in gas phase:
    x_g_H2_in  = (pressure_g_in - pressureCH4resid) / pressure_g_in * x_g_H2_th
    x_g_CO2_in = (pressure_g_in - pressureCH4resid) / pressure_g_in * x_g_CO2_th 
    x_g_CH4_in = 1 - x_g_H2_in - x_g_CO2_in
   
            # Initial molecular weight of gasses:
    M_g = (x_g_H2_in * M_H2) + (x_g_CO2_in * M_CO2) + (x_g_CH4_in * M_CH4)

            #Initial gas density:
    R_gas_in = 8.314/M_g
    density_g_in = pressure_g_in/R_gas_in/temperature

            # Initial mass fraction of components in gas phase:
    w_g_H2_in  = (x_g_H2_in * M_H2) / M_g
    w_g_CO2_in = (x_g_CO2_in * M_CO2) / M_g
    w_g_CH4_in = (x_g_CH4_in * M_CH4) / M_g
    
            # Time array in seconds and end time:
    time_sim = np.array(time) - np.array(time[0])
    time_sim = time_sim * 24 * 60 * 60               
    time_end = time_sim[-1]
    
           # Initial values for the parameters of interest - except biomass:
    U_H2 = 10**theta[1]
    K_Monod_H2 = 10**theta[2]
    K_Monod_CO2 = 10**theta[3]
    d = 10**theta[4]
    x_l_eps = 10**theta[5]
    
            # Solve_ivp to solve for RK45():
    sol = solve_ivp(rk45, [0,time_end],y0=[w_g_H2_in, w_g_CO2_in, w_g_CH4_in, w_l_H2_in, w_l_CO2_in, w_l_CH4_in_new, BM_in_new, density_g_in, S_liq_in], 
                    args=(U_H2, K_Monod_H2, K_Monod_CO2, d, x_l_eps), t_eval=time_sim, atol=1e-8, rtol=1e-6)
    
            # Save the tracked and returned values from solve_ivp():   
    w_g_H2 = sol.y[0]
    w_g_CO2 = sol.y[1]
    w_g_CH4 = sol.y[2]
    w_l_H2 = sol.y[3] 
    w_l_CO2 = sol.y[4]
    w_l_CH4 = sol.y[5]
    biomass = sol.y[6]
    density_g = sol.y[7]
    S_liq = sol.y[8]
    
            # Calculate the new initial saturation of liquid phase to use as input for the next fill:
    S_liq_in_new = S_liq[-1] * (1 - (1 - 0.557) * w_l_CH4[-1] - w_l_H2[-1] - w_l_CO2[-1])
            
            # Calculate the new initial molecular weight of gasses:
    M_g_new = ((w_g_H2 / M_H2) + (w_g_CO2 / M_CO2) + (w_g_CH4 / M_CH4))**-1
    
            # Calculate the pressure change over time: 
    R_gas = 8.314 / M_g_new
    pressure_g_bar = density_g * R_gas * temperature * 1e-5 #to change from pascal to bar * 1e-5
    pressure_g_pascal = density_g * R_gas * temperature

            # Calculate the initial mass fraction of CH4 to input into next fill:    
    w_l_CH4_in_new = w_l_CH4[-1]*0.557 # 55.7% of the end value go to the next fill

            # Calculate the initial biomass to input into next fill:
    BM_in_new = biomass[-1] / (1 - (1 - 0.557) * w_l_CH4[-1] - w_l_H2[-1] - w_l_CO2[-1])

            # Variables to be returned:
    return w_l_CH4_in_new, pressure_g_pascal, S_liq_in_new, BM_in_new, biomass

In [ ]:
#Generate pressure_g_sim -> simulated pressure

def simulated_pressure(df_time, in_pressure, theta0):
    pressure = []
    biomass = []
    pressure_g_sim_concatenated = []
    biomass_sim = []
    w_l_CH4_end_all = []
    S_liq_end_all = []
    
    for i in range(1):
        [w_l_CH4_in, pressure_g_sim_fill1, S_liq_in, BM_in, biomass_fill1] = pressure_fill1(df_time[0], in_pressure[0], theta0)
        pressure.append(pressure_g_sim_fill1)
        biomass.append(biomass_fill1)
        w_l_CH4_end_all.append(w_l_CH4_in)
        S_liq_end_all.append(S_liq_in)
        
    for i in range(1, len(df_pressure)):
        [w_l_CH4_in, pressure_g_sim_fills, S_liq_in, BM_in, biomass_fills] = pressure_otherfills(df_time[i], in_pressure[i], theta0, w_l_CH4_in, S_liq_in, BM_in)
        pressure.append(pressure_g_sim_fills)
        biomass.append(biomass_fills)
        w_l_CH4_end_all.append(w_l_CH4_in)
        S_liq_end_all.append(S_liq_in)
    
    gas_pressure = np.concatenate(pressure[0:len(pressure)])
    pressure_g_sim_concatenated.append(gas_pressure)
    pressure_g_sim_concatenated = pressure_g_sim_concatenated[0]
    
    biomass_concatenated = np.concatenate(biomass[0:len(biomass)])
    biomass_sim.append(biomass_concatenated)
    biomass_sim = biomass_sim[0]
    biomass_endvalue = biomass_sim[-1]


    w_l_CH4_endvalue = w_l_CH4_end_all[-1]
    
    S_liq_endvalue = S_liq_end_all[-1]
    
    return pressure, biomass, biomass_endvalue, biomass_concatenated, w_l_CH4_endvalue, S_liq_endvalue
    
    
[pressure_g_sim, biomass, biomass_endvalue, biomass_concatenated, w_l_CH4_endvalue, S_liq_endvalue] = simulated_pressure(df_time, in_pressure, theta0)

In [ ]:
# Print end values of biomass, CH4 mass fraction (w_l_CH4), and liquid saturation (S_liq)

print("w_l_CH4 end value:", "{:.2e}".format(w_l_CH4_endvalue))
print("S_liq end value:", "{:.2e}".format(S_liq_endvalue))
print("Biomass end value:", round(biomass_endvalue,2))

# Plots

In [ ]:
def plot_biomass(df_time, biomass):

    fig, (ax1) = plt.subplots(figsize=(12,6))
    
    ax1.plot(df_time[0], biomass[0], color="green", label='Biomass')

    ax1.legend(loc="lower right")
    ax1.set_xlabel("Days")
    ax1.set_ylabel("Biomass [Kg/m3]")
    
    for i in range(1, len(biomass)):

            ax1.plot(df_time[i], biomass[i], color="green", label='_nolegend_')


    ax1.set_title("Reactor 1 - H2 Limiting Model - 10% H2 and 2.5% CO2 Gas Composition - Biomass Change Over Time ")
    
plot_biomass(df_time, biomass)

In [ ]:
# Generate experimental pressure in pascals

exp_pressure = [[] for i in range(len(df_pressure))]

def experimental_pressure(df_pressure):
    
    for i in range(len(df_pressure)):

        for pressure in df_pressure[i]:
            pres = pressure * 1e5
            exp_pressure[i].append(pres)
    return exp_pressure

exp_pressure = experimental_pressure(df_pressure)


In [ ]:
def plot_pressure_comparison(df_time, df_pressure, pressure_g_sim):

    fig, (ax1) = plt.subplots(figsize=(12,6))
    
    ax1.plot(df_time[0], pressure_g_sim[0], color="brown", label='Simulated Pressure')
    ax1.plot(df_time[0], df_pressure[0], color="cornflowerblue", label='Experimental Pressure')
    ax1.legend(loc="upper right")
    ax1.set_xlabel("Days")
    ax1.set_ylabel("Pressure [Pa]")
    
    for i in range(1, len(pressure_g_sim)):

            ax1.plot(df_time[i], pressure_g_sim[i], color="brown", label='_nolegend_')

            ax1.plot(df_time[i], df_pressure[i], color="cornflowerblue", label='_nolegend_')


    ax1.set_title("Reactor 1 - H2 Limiting Model - 10% H2 and 2.5% CO2 Gas Composition - Pressure Comparison")
    
plot_pressure_comparison(df_time, exp_pressure, pressure_g_sim)

# - Pressure difference function

In [ ]:
# Generate the pressure difference between experimental and simulated values

def pressure_diff(theta0):

    [pressure_g_sim, biomass, biomass_endvalue, biomass_concatenated, w_l_CH4_endvalue, S_liq_endvalue] = simulated_pressure(df_time, in_pressure, theta0)
    
    sim_pressure = np.concatenate(pressure_g_sim[0:len(pressure_g_sim)])
    exp_pressure = np.concatenate(df_pressure[0:len(df_pressure)])
    
    diff = exp_pressure - sim_pressure

    return diff

pressure_g_diff = pressure_diff(theta0)

# - Random theta0 and optimal values function

In [ ]:
# Generate a number of random theta0 values to use as initial guesses for the otpimal_values() function

            # Set upper and lower bounds for the random theta0 values
u_bound = [2, 9, -2, -2, -2, -6]
l_bound = [-6, 3, -8, -8, -5, -10]
            
            
theta0_random = []

def random_theta0(lower_bound, upper_bound, samples):
            
    for i in range(samples):
        theta0 = [None] * 6

        for i in range(len(theta0)):
            theta0[i] = random.uniform(lower_bound[i], upper_bound[i])
        theta0_random.append(theta0)
    
            
random_theta0(l_bound, u_bound, 10) # <- Change accordingly as needed

In [ ]:
# Look for optimal values for the random theta0 values and generate norm of residuals for the optimization

norm_res = []
optimal_value = []

def optimal_values(theta0_random):
    
    for i in range(len(theta0_random)):
        res = least_squares(pressure_diff, theta0_random[i], bounds=(l_bound, u_bound))
        optimal = res.x
        optimal_value.append(optimal)
        
        residuals = res.fun
        norm = np.linalg.norm(residuals)
        norm_res.append(norm)

optimal_values(theta0_random)

In [ ]:
# Create a dataframe (and write to .csv) of the generated optimal values, the corresponding random theta0 values and the generated norm of residuals

            # Create a dictionary with the above generated values:
dict = {'theta0': theta0_random, 'optimal_values': optimal_values, 'norm_res': norm_res} 

            # Convert the dictionary to a pandas dataframe and write to .csv format:
dc = pd.DataFrame(dict) 
dc.to_csv('optimal_values_reactor2UH2.csv')